# **pytorch 实现 skip-gram**




In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import numpy as np
import math
import random
import pandas as pd
import scipy

# 固定seed
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1)

# 设定超参数
C = 3 #context window size
K = 2 #number of negative samples 
NUM_EPOCHS = 1 #运行太慢了，这里设为1
VOCAB_SIZE = 30000
BATCH_SIZE = 128
LEARNING_RATE = 0.1
EMBEDDING_SIZE = 100


从文本文件中读取所有的文字，通过这些文本创建一个vocabulary

由于单词数量可能太大，我们只选取最常见的VOCAB_SIZE个单词

我们添加一个UNK单词表示所有不常见的单词

需记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [0]:
with open("text8.train.txt","r") as fin:
    text = fin.read()

text = text.split() #分词，以最简单的空格进行分词

# 构建词汇表

#Counter会统计每个单词的次数
#mostcommon（m）找出出现次数最多的m个单词作为词库，-1是因为要分配给UNK一个
vocab = dict(Counter(text).most_common(VOCAB_SIZE-1))
#剩下的词看作不常出现的单词，计算这些单词次数(所有单词出现的次数-词库中每个单词出现的次数的和)
vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))
#vocab中记录了每个单词及其出现的次数

idx_to_word = {index:word for index,word in enumerate(vocab)}
word_to_idx = {word:index for index,word in idx_to_word.items()}
# print(list(idx_to_word.items())[:10])
# print(list(word_to_idx.items())[:10])

In [0]:
# 计算word frequency以便用于负采样
word_counts = np.array([count for count in vocab.values()],dtype=np.float32) #m每个单词出现的频次count
word_frequencys = word_counts / np.sum(word_counts) #每个单词出现的频率 
word_frequencys = word_frequencys **(3.0/4.0) #论文中提到的3/4
word_frequencys = word_frequencys / np.sum(word_frequencys) #实际上是记录了每个单词被采样的权重（可看作概率）




## 实现Dataloader

一个dataloader需要以下内容：

把所有text编码成数字

保存vocabulary，单词count，normalized word frequency

每个iteration sample一个中心词

根据当前的中心词返回context单词

根据中心词sample一些negative单词


这里有一个好的tutorial介绍如何使用PyTorch dataloader. 为了使用dataloader，我们需要定义以下两个function:

$$__len__(self)需要返回整个数据集中有多少个item$$

$$__getitem__(self,index) 根据给定的index返回一个item$$

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

In [0]:
import torch.utils.data as tud

class WordEmbeddingDataset(tud.Dataset):
    def __init__(self,text,word_to_idx,idx_to_word,word_frequencys,word_counts):
        #要记录这些信息
        super(WordEmbeddingDataset,self).__init__()
        #将woed用数字表示
        self.text_encoded = [word_to_idx.get(word, word_to_idx["<unk>"]) for word in text]
        #dict.get(key, default=None),返回key对应的value，如哦value不存在，返回default设定的值

        self.text_encoded = torch.LongTensor(self.text_encoded)
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.word_frequencys = torch.Tensor(word_frequencys)
        self.word_counts = torch.Tensor(word_counts)
    def __len__(self):
        #返回这个数据集一共多少个item
        return len(self.text_encoded)
    def __getitem__(self,idx):
        #通过__getitem__函数获取单个的数据，然后组合成batch
        center_word = self.text_encoded[idx] # 中心词的id
        pos_indices = list(range(idx-C,idx)) + list(range(idx+1,idx+C+1))#中心词的前c个和后c个
        #处理边界情况，idx-C可能<0 or idx+C>len
        pos_indices = [i % len(self.text_encoded) for i in pos_indices]
        pos_words = self.text_encoded[pos_indices]#hang list[list]?答lsit[list]不支持，tensor[lsit]支持

        #negative words
        #multinomial 多项式分布概率采样(input, num_samples, replacement=False)
        #input张量可以看成一个权重张量，每一个元素代表其在该行中的权重
        #如果有元素为0，那么在其他不为0的元素，被取干净之前，这个元素是不会被取到的。
        #返回其索引，若input有m行，则抽取m乘numsamples个，True表示有放回的采样
        neg_words = torch.multinomial(self.word_frequencys, K*pos_words.shape[0], False)
        # hang freq？不应该是正样本以外的样本中抽取吗，对每个上下文词都要抽取K个负样本
        #答：multinomial是根据权重抽取其对应的索引

        return center_word, pos_words, neg_words

In [0]:
dataset = WordEmbeddingDataset(text,word_to_idx,idx_to_word,word_frequencys,word_counts)
dataloader = tud.DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)#numworkers好像是指以4个线程启动

## 定义pytoch模型


In [0]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(EmbeddingModel, self).__init__()
        initrange = 0.5 / embed_dim #可选
        self.in_embed = nn.Embedding(vocab_size,embed_dim)
        self.in_embed.weight.data.uniform_(-initrange,initrange) #可选
        self.out_embed = nn.Embedding(vocab_size,embed_dim)
        
    def forward(self,input_labels,pos_labels,neg_labels):
        """
        input_labels: 输入的中心词, [batch_size]，就是单词的id
        pos_labels: 中心词周围 的单词 [batch_size，(window_size*2)]每个中心词对应windowsszie*2个正样本词
        neg_labelss: [batch_size, (window_size*2*K)]#对每个pos词，都要采K个负样本
        
        return: loss, [batch_size]
        """
        #embedding层的输入是整数索引，即单词的index
        input_embedding = self.in_embed(input_labels) #【batchsize, embeddim】
        pos_embedding = self.in_embed(pos_labels)#【batch_size，(window_size*2)，embeddim】
        neg_embedding = self.in_embed(neg_labels)#【batch_size, (window_size*2*K)，embeddim】

        #需要将inputembedding和posembedding做点积
        input_embedding = input_embedding.unsqueeze(2)#在第w维加上1维，【batchsize, embeddim，1】

        pos_dot = torch.bmm(pos_embedding,input_embedding)#【batch_size，(window_size*2)，1】
        #input是【b，n，m】tensor, mat2是【b，m，p】tensor,torch.bmm（input,mat2）得到【b，n，p】tensor.

        pos_dot = pos_dot.squeeze(2)#去掉第2维的1，【batch_size，(window_size*2)】

        neg_dot = torch.bmm(neg_embedding,input_embedding)#【batch_size，(window_size*2*K)，1】
        neg_dot = neg_dot.squeeze(2)#去掉第2维的1，【batch_size，(window_size*2*K)】

        log_pos = F.logsigmoid(pos_dot).sum(1)#第一维是batch
        log_neg = F.logsigmoid(-neg_dot).sum(1) #hang 关于logsigmoid和n-neg_dot见笔记，作者这里没有加负号

        loss = log_pos + log_neg
        
        return -loss #求argmin.【batchsize】
    
    def input_embedding(self):#获取embedding
        return self.in_embed.weight.data.cpu().numpy()



**定义模型以及将模型移动到GPU**

每个epoch我们都把所有的数据分成若干个batch

把每个batch的输入和输出都包装成cuda tensor

forward pass，通过输入的句子预测每个单词的下一个单词

用模型的预测和正确的下一个单词计算cross entropy loss

清空模型当前gradient

backward pass

更新模型参数

每隔一定的iteration输出模型在当前iteration的loss，以及在验证数据集上做模型的评估


In [63]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_SIZE)
if torch.cuda.is_available():
    model = model.cuda()
#lossfn = #这里不需要了，因为已经在model的forward定义了loss
#hang  那loss.backward（）怎么办？
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

for epochIndex in range(NUM_EPOCHS):
    print("epoch:",epochIndex)
    for i,(input_labels,pos_labels,neg_labels) in enumerate(dataloader):#对每个batchsize

        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        if torch.cuda.is_available():
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()

            optimizer.zero_grad()
            loss = model(input_labels,pos_labels,neg_labels).mean() #model.forward()
            #注意loss应该是a tensor with single number，我们的是【batchsize】大小的tensor，所以做个mean
            loss.backward()
            optimizer.step()

        if i % 1000 == 0:
            print("batch:",i,loss.item())
    break            
    if epochIndex %10 == 0:
        print("epoch:",epochIndex, loss.item())


epoch: 0
batch: 0 12.476619720458984
batch: 1000 12.44610595703125
batch: 2000 12.291523933410645
batch: 3000 12.357091903686523
batch: 4000 12.278715133666992
batch: 5000 12.131007194519043
batch: 6000 12.270715713500977
batch: 7000 12.223976135253906
batch: 8000 12.267290115356445
batch: 9000 12.310798645019531
batch: 10000 12.184684753417969
batch: 11000 12.246097564697266
batch: 12000 12.23458194732666
batch: 13000 12.22145938873291
batch: 14000 12.235976219177246
batch: 15000 12.13444995880127
batch: 16000 12.193893432617188
batch: 17000 12.246011734008789
batch: 18000 12.132369995117188
batch: 19000 12.228584289550781
batch: 20000 12.10348892211914
batch: 21000 12.113518714904785
batch: 22000 12.120660781860352
batch: 23000 12.272673606872559
batch: 24000 12.20083236694336
batch: 25000 12.30928897857666
batch: 26000 12.189101219177246
batch: 27000 12.185226440429688
batch: 28000 12.111093521118164
batch: 29000 12.210922241210938
batch: 30000 12.121561050415039
batch: 31000 12.182

KeyboardInterrupt: ignored

# 评估模型

 - 计算斯皮尔曼系数

 - find最近的单词

In [64]:
import scipy.spatial

#获得所有单词的embedding
embedding_weights = model.input_embedding()

def find_nearest(word):
    #与所有单词的embedding向量计算余弦距离，返回最小的5个单词
    word_id = word_to_idx[word]
    embedding = embedding_weights[word_id]
    cos_distance = np.array([scipy.spatial.distance.cosine(e,embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_distance.argsort()[:5]]

for word in ["good", "green", "like", "work", "computer"]:
    print(word, find_nearest(word))

good ['good', 'simple', 'technique', 'very', 'device']
green ['green', 'blue', 'etc', 'binomial', 'rice']
like ['like', 'plural', 'disease', 'singular', 'extinct']
work ['work', 'his', 'her', 'life', 'home']
computer ['computer', 'natural', 'analysis', 'variant', 'discussion']


## 单词之间的关系

man-king = women-queen

In [65]:

man_idx = word_to_idx["man"] 
king_idx = word_to_idx["king"] 
woman_idx = word_to_idx["woman"]
embedding = embedding_weights[woman_idx] - embedding_weights[man_idx] + embedding_weights[king_idx]
#在所有单词的embedding中找出与这个embedding最近的单词
cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
for i in cos_dis.argsort()[:10]:
    print(idx_to_word[i])

king
emperor
president
st
queen
duke
prince
alexander
henry
ii
